# Double-q learning
An example

In [14]:
import agents
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

In [2]:
%config InlineBackend.figure_format = "retina"

In [4]:
map_states = {
    "A": 0,
    "B": 1,
}

map_actions = {
    "left": 0,
    "right": 1,
}

In [7]:
n_arms_B = 10
rewards = np.random.normal(loc=-1, scale=1, size=n_arms_B)

In [30]:
@njit
def step(state, action):
    if state == 0:
        reward = 0
        if action == 1: # right
            new_state = 0
        if action == 0: # left
            new_state = 1

    if state == 1:
        new_state = 0
        reward = rewards[action]

    return new_state, reward

In [12]:
Qv = (
    np.zeros(2,),
    np.zeros(n_arms_B,)
)

In [26]:
epsilon = 0.1
state = 1
agents.choose_action(state, Qv, epsilon)

5

In [ ]:
def q_step(state, Q, epsilon):
    action = agents.choose_action(state, Q)
    new_state, reward = step(state, action)
    q_new = Q[state][action] + alpha * (reward + gamma * Q[state].max() - Q[state][action])
    return q_new